In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy


# SEED = np.random.randint(100)
np.random.seed(53)

In [3]:
data1 = pd.read_csv('CA2Data/animals', header=None, delimiter=' ')
data2 = pd.read_csv('CA2Data/countries', header=None, delimiter=' ')
data3 = pd.read_csv('CA2Data/fruits', header=None, delimiter=' ')
data4 = pd.read_csv('CA2Data/veggies', header=None, delimiter=' ')
data1[0] = 1
data2[0] = 2
data3[0] = 3
data4[0] = 4

dataset = pd.concat([data1, data2, data3, data4])
# print(data1.shape, data2.shape, data3.shape, data4.shape)
# dataset = pd.concat([data1.iloc[:,1:], data2.iloc[:,1:], data3.iloc[:,1:], data4.iloc[:,1:]])

In [4]:
dataset = dataset.reset_index(drop=True)
# dataset.insert(loc=0, column=301, value=dataset.index)
index_data = pd.DataFrame(dataset, columns=[0])
index_data['index'] = index_data.index

dataset.drop(columns=[0], inplace=True)
dataset
index_data

,0,index
0,1,0
1,1,1
2,1,2
3,1,3
4,1,4
...,...,...
322,4,322
323,4,323
324,4,324
325,4,325


In [5]:
dataset = np.array(dataset)
dataset.shape
len(dataset)

327

In [6]:
# K-mean * K-medians

def euclidean_distance(X,Y):
    # Return the Euclidean distance between X and Y
    return np.linalg.norm(X-Y)


def manhattan_distance(X,Y):
    # Return the Manhattan distance between X and Y
    # return np.abs(X-Y).sum()
    return np.sum(np.abs(X-Y))


def check_convergence(k_means, k_medians):

    # if k_means:
    #     np.sum()
    pass


def get_centroid_positions(k_means, k_medians):
    # Initialise 'clusters', to later store the centroid index value to the corresponding datapoint index
    clusters = np.zeros((len(dataset),1))
    num_datapoints = len(dataset)

    # loop through each datapoint
    for index in range(num_datapoints):
        # Initialise 'distances', to store the 'distance' value from each centroid to the datapoint
        distances = np.zeros((num_centroids, 1))
        # Loop through each centroid
        for centroid_index in range(num_centroids):
            if k_means:
                # for k-means, find the eucledian distance from the datapoint to each centroid
                distance_to_centroid = euclidean_distance(dataset[index, :], centroids[centroid_index, :])
                # update the 'distance' value to the 'distances' array
                distances[centroid_index] = distance_to_centroid

            elif k_medians:
                # for k-medians, find the manhattan distance from the datapoint to each centroid
                distance_to_centroid = manhattan_distance(dataset[index, :], centroids[centroid_index, :])
                # update the 'distance' value to the 'distances' array
                distances[centroid_index] = distance_to_centroid

        # Get the closest centroids index value
        closest_centroid_index = np.argmin(distances)
        # Assign the closest centroid index value to the corresponding datapoint index (in the 'clusters' array)
        clusters[index] = closest_centroid_index
#     print(clusters)
    return clusters

In [7]:

import pandas as pd
import numpy as np
from copy import deepcopy

SEED = np.random.randint(100)
np.random.seed(53)
# print(SEED)
# 53, 40, 18

# import data
data1 = pd.read_csv('CA2Data/animals', header=None, delimiter=' ')
data2 = pd.read_csv('CA2Data/countries', header=None, delimiter=' ')
data3 = pd.read_csv('CA2Data/fruits', header=None, delimiter=' ')
data4 = pd.read_csv('CA2Data/veggies', header=None, delimiter=' ')
# dataset = pd.concat([data1, data2, data3, data4])

# concatenate data points
dataset = pd.concat([data1.iloc[:,1:], data2.iloc[:,1:], data3.iloc[:,1:], data4.iloc[:,1:]])

# convert data set to numpy array
dataset = np.array(dataset)

# choose 'k' for number of clusters 
k=4
num_centroids = k

# Initialise 'k' centroids (y1, .. yk) randomly from the data set
centroids = dataset[np.random.randint(dataset.shape[0], size=num_centroids), :]

# To only run K_means
k_means = True
k_medians = False

# To only run K_medians
# k_means = False
# k_medians = True


#### update centroids based on new cluster data
for _ in range(1000):
    # if check_convergence():
    #     break
    num_centroids = k

    # for the current iteration, get the centroids (index value) assigned to the corresponding datapoints index
    clusters = get_centroid_positions(k_means, k_medians)
    data = deepcopy(dataset)

    # Iterate over the centroids, to update them based on the updated cluster data
    for centroid_index in range(num_centroids):
        if k_means:
            # Compute the mean of each cluster, and set them as the new centroids
            centroids[centroid_index] = np.mean(data[clusters.flatten() == centroid_index], axis=0)
        elif k_medians:
            # Compute the median of each cluster, and set them as the new centroids
            centroids[centroid_index] = np.median(data[clusters.flatten() == centroid_index], axis=0)

print("\nKMeans Centroids from computed for k = 4:")
print(centroids)

#### ******************* IGNORE CODE BELOW ********************

#### TESTING THE CODE

from sklearn.cluster import KMeans

model = KMeans(n_clusters=k)
model.fit(data)
print("\n\nOutput from SKLEARN")
print(model.cluster_centers_)


KMeans Centroids from computed for k = 4:
[[-0.07048999 -0.20060509 -0.26966388 ... -0.1941668   0.06881185
   0.18472587]
 [-0.11743746 -0.11042181 -0.0659227  ... -0.10309246 -0.21821709
   0.1395143 ]
 [-0.23255197 -0.09005632  0.09137848 ... -0.44305674 -0.06855837
   0.05151326]
 [ 0.38627178  0.3143458   0.05909229 ...  0.09175967  0.11883499
   0.25313992]]


Output from SKLEARN
[[-0.23340746 -0.0955901   0.07988927 ... -0.43806341 -0.06295059
   0.05672025]
 [ 0.38627178  0.3143458   0.05909229 ...  0.09175967  0.11883498
   0.25313992]
 [-0.10709906 -0.10474646 -0.06334103 ... -0.08248652 -0.23759206
   0.13968116]
 [-0.07048999 -0.20060509 -0.26966388 ... -0.1941668   0.06881185
   0.18472587]]


In [8]:
d = np.array([[1,2,3], 
              [2.3,5.2,6],
              [9,11,7]])
print(d)
np.median(d, axis=0)

[[ 1.   2.   3. ]
 [ 2.3  5.2  6. ]
 [ 9.  11.   7. ]]


array([2.3, 5.2, 6. ])